In [39]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt

In [48]:
select = 'valid'
iou_threshold = 0.5

GT_JSON = f'../../../Split_data/{select}_3_5.json'
PRED_CSV = f'./result/deta-swin-large_3_img_size_720_valid.csv'

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal",
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]




In [49]:
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

pred = []
file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).strip().split(' '))

    if len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    elif isinstance(bbox, float):
        print(f'{file_name} empty box')
        continue
    else:
        raise Exception('error', 'invalid box type')
    
    for box in boxes:
        pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

958it [00:00, 5203.37it/s]


In [50]:
gt = []
coco = COCO(GT_JSON)

for image_id in coco.getImgIds():
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
    file_name = image_info['file_name']

    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                float(annotation['bbox'][0]),
                float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                float(annotation['bbox'][1]),
                float(annotation['bbox'][1]) + float(annotation['bbox'][3])])
        
mean_ap, average_precisions = mean_average_precision_for_boxes(gt, pred, iou_threshold=iou_threshold)
print(f'mAP@{iou_threshold*100:.0f}: {mean_ap}')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Number of files in annotations: 958
Number of files in predictions: 958
Unique classes: 10
Detections length: 958
Annotations length: 958
0                              | 0.422468 |     804
1                              | 0.616130 |    1214
2                              | 0.672288 |     180
3                              | 0.646937 |     190
4                              | 0.786938 |     229
5                              | 0.586005 |     595
6                              | 0.560607 |     328
7                              | 0.766959 |     941
8                              | 0.861881 |      51
9                              | 0.741529 |      55
mAP: 0.666174
mAP@50: 0.6661741587694356
